In [4]:
import nltk
import os 
import glob
import re
import  pickle
from nltk.tokenize import sent_tokenize, word_tokenize
from functools import partial
from multiprocessing import Pool

In [5]:
def useless_words(delete_words,target):
    for w in delete_words:
        if w in target:
            return False
    return True

def process_documents(f):
    index, file = f 
    with open(file,'r',encoding='utf-8') as f:
        text = f.read()
    text = replace_regex.sub(' ',text)
    text = re.sub(r'\(.*?\d+.*?\)|\s\s\s\s+\d','',text)  # clean up things like '(box 1)' and  '     1'
    sentances = sent_tokenize(text)
    
    clean_sentances= list()
    for line in sentances:
        if useless_words(delete_words,line):
            tokens = word_tokenize(line)
            if len(tokens)>5:
                wl = [len(t) for t in tokens]
                if max(wl)<30:
                    line = number_regex.sub(' ',line)
                    tokens = word_tokenize(line.lower())
                    clean_sentances.append(tokens)
    if index%1000 == 0: print('finished ', index)
    return clean_sentances

In [6]:
if __name__ =='__main__':
    
    ## run a small sample of test first 
    test = False
    
####################################
## set up some global variables 
####################################

    delete_words = ['____','DOCUMENT OF INTERNATIONAL MONETARY FUND','Download Date']
    replace_regex = re.compile(r'\uf0b7|\x0c')
    number_regex = re.compile(r'\d+?\.\d+|\.\d+|\d+-\d+|\d+/\d+|\d+/|-\d+|\d+|½|¼|¼|⅔|¾/') ## clean up all kinds of numbers
    #test = '123.45 123.25 1234 .245 2015-218  -60 1/asdf 2/asdf  2021/22 ////'
    #replace_regex.findall(test)

    files_path = 'txt_files_finished/'
    files = sorted(glob.glob(files_path+'*.txt'))
    if len(files) == 0:
        raise ValueError('could not find input files')
        
###############################
## run text process in parellel
###############################
    ## multi process it 
    num_cores = os.cpu_count()
    
    ## see if we want to run a test first
    if test == True: 
        files = files[:100]
        num_cores = 2
    else:
        num_cores = os.cpu_count()
    
    #multi process unpacking
    p = Pool(num_cores)
    partial_unpack = partial(process_documents)
    process_mp = p.map(partial_unpack,enumerate(files))
    p.close()
    p.join()
    print('finish')



finished  0
finished  3800
finished  2700
finished  3300
finished  3900
finished  3400
finished  1100
finished  4000
finished  3500
finished  2200
finished  2800
finished  4100
finished  600
finished  4200
finished  3600
finished  2900
finished  3700
finished  4300
finished  3000
finished  4400
finished  1700
finished  4900
finished  3100
finished  4500
finished  2300
finished  5000
finished  4600
finished  3200
finished  5100
finished  100
finished  4700
finished  5400
finished  5200
finished  1200
finished  5500
finished  4800
finished  5300
finished  2400
finished  700
finished  5600
finished  2500
finished  6500
finished  1800
finished  5900
finished  200
finished  6600
finished  2600
finished  800
finished  1300
finished  5700
finished  6700
finished  7000
finished  7100
finished  6800
finished  5800
finished  7200
finished  1900
finished  6900
finished  7500
finished  6000
finished  900
finished  7300
finished  300
finished  2000
finished  1400
finished  7400
finished  2100
finis

In [12]:
total_results = [l for sublist in process_mp for l in sublist]
pickle.dump(total_results,open("sentances.p", "wb"))
#total_results = pickle.load(open("sentances.p", "rb"))